In [1]:
import pandas as pd
import numpy as np
import os 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score

In [2]:
#afficher le répertoire courant
os.getcwd()
#Changer de répertoire de travail
os.chdir('/home/senessou')

In [3]:
#Chargement du DF
data = pd.read_excel('Copie de Coeur.xlsx')

In [4]:
#Travailler avec une copie du data Frame pour éviter de le rechager en cas d'êrreur
df = data.copy()

In [5]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [6]:
#Normalisation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = df[col]-df[col].mean()/df[col].std()


In [7]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,homme,AA,132.848903,287.182556,-0.551041,Normal,166.626568,Non,-0.831979,Ascendant,0
1,43.327036,femme,DNA,152.848903,178.182556,-0.551041,Normal,150.626568,Non,0.168021,Plat,1
2,31.327036,homme,AA,122.848903,281.182556,-0.551041,ST,92.626568,Non,-0.831979,Ascendant,0
3,42.327036,femme,ASY,130.848903,212.182556,-0.551041,Normal,102.626568,Oui,0.668021,Plat,1
4,48.327036,homme,DNA,142.848903,193.182556,-0.551041,Normal,116.626568,Non,-0.831979,Ascendant,0


In [8]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [9]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,1,0,132.848903,287.182556,-0.551041,1,166.626568,0,-0.831979,0,0
1,43.327036,0,3,152.848903,178.182556,-0.551041,1,150.626568,0,0.168021,2,1
2,31.327036,1,0,122.848903,281.182556,-0.551041,2,92.626568,0,-0.831979,0,0
3,42.327036,0,1,130.848903,212.182556,-0.551041,1,102.626568,1,0.668021,2,1
4,48.327036,1,3,142.848903,193.182556,-0.551041,1,116.626568,0,-0.831979,0,0


In [10]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [11]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [12]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [13]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)

In [14]:
#Apprentissage du modèle
model = lr.fit(x_train, y_train)
#model = lr.fit(train_set.drop('CŒUR', axis = 1), train_set['CŒUR'])

In [15]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.34770612, 0.65229388],
       [0.17508424, 0.82491576],
       [0.03708029, 0.96291971],
       [0.08135272, 0.91864728],
       [0.18111693, 0.81888307]])

In [16]:
#Application du modèle au données de test
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 1])

In [17]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  8, 102]])

In [18]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 85.32 % de bonne prédiction.

0.9130434782608695

In [19]:
#Sensibilité
recall = recall_score(y_test, y_pred)
recall
#conclusion : lorsqu'un individu est malade, nous avons 84.11 % de chance que le modèle le prédise.

0.9272727272727272

In [20]:
#Précision
precision = precision_score(y_test, y_pred)
precision
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90 % de chace que cela soit vraie.

0.9272727272727272

In [21]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        74
           1       0.93      0.93      0.93       110

    accuracy                           0.91       184
   macro avg       0.91      0.91      0.91       184
weighted avg       0.91      0.91      0.91       184



In [22]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [23]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8460490463215259

In [24]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model.score(x_test, y_test)

0.9130434782608695

# Arbre de decision

In [25]:
#Subdivision du jeu de données en apprentissage et en test
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =300, random_state =1)

In [26]:
#Vérifier le résultat
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [27]:
#instanciation de l'arbre
arbreFirst = DecisionTreeClassifier()

In [28]:
#construction de l'arbre
model2 = arbreFirst.fit(x_train, y_train)

In [29]:
#prédiction sur l'échantillon test
# predFirst = arbreFirst.predict(x_test)
predFirst = model2.predict(x_test)
predFirst[:5]

array([1, 0, 1, 1, 0])

In [30]:
#distribution des predictions
# print(np.unique(predFirst,return_counts=True))

In [31]:
#matrice de confusion
mc = confusion_matrix(y_test, predFirst)
mc

array([[64, 10],
       [31, 79]])

In [32]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, predFirst)
accuracy
#Conclusion : le modèle produit 85.32 % de bonne prédiction.

0.7771739130434783

In [33]:
#Sensibilité
recall = recall_score(y_test, predFirst)
recall
#conclusion : lorsqu'un individu est malade, nous avons 84.11 % de chance que le modèle le prédise.

0.7181818181818181

In [34]:
#précision
precise = precision_score(y_test,predFirst)
precise
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90 % de chace que cela soit vraie.

0.8876404494382022

In [35]:
#plus de détails avec classification report
cr = classification_report(y_test, predFirst)
print(cr)

              precision    recall  f1-score   support

           0       0.67      0.86      0.76        74
           1       0.89      0.72      0.79       110

    accuracy                           0.78       184
   macro avg       0.78      0.79      0.78       184
weighted avg       0.80      0.78      0.78       184



In [36]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [37]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model2.score(x_train, y_train)

1.0

In [38]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model2.score(x_test, y_test)

0.7771739130434783